# Retrieve と RetrieveAndGenerate の違い

Knowledge bases for Amazon Bedrock を利用した検索には 2 種類の API が提供されています。

- [Retrieve](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_Retrieve.html)
- [RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html)

文字通り Retrieve API はベクトルデータベースから情報の取得のみを行い、 RetrieveAndGenerate API はベクトルデータベースから情報の取得した後、回答生成まで行います。

<img src="../images/01/01.png">